In [1]:
import numpy as np
import torch

In [2]:
device ='cpu' 

### load data and configurations

In [3]:
import utils_data as ud

x,xf,tr,st= ud.load_data('data/x_xf_tr_st_NO2.p',device)
cfg, mean, std, station_mean, station_std= ud.load_configurations()
min_data,max_data= ud.load_minmax()
##print station numbers

0: WESE, 1: RODE, 2: SOES, 3: MSGE, 4: WALS, 5: HUE2, 6: LOER, 7: DMD2, 8: WULA, 9: STYR, 10: EIFE, 11: LEV2, 12: SOLI, 13: BOTT, 14: SHW2, 15: BORG, 16: CHOR, 17: ROTH, 18: NIED, 19: AABU, 20: RAT2, 21: BIEL, 

#### select hour of prediction 

In [4]:
hour=5
inp_p = x[[hour],:,:]   
inp_f = xf[[hour],:,:]   
targs = tr[[hour],:,:]   
stat=st[[hour],:]   
inp_p.shape

torch.Size([1, 168, 71])

### load model parameters 

In [5]:
import sys
sys.path.append('../LRP')

In [6]:
import utils_model as um

loading_path='model/no_binarystd_GRUh1024l2dr0.3O3v0/last_checkpoint'
wi_tuples,wh_tuples,bi_tuples,bh_tuples,wl,bl= um.model_parameters(loading_path,device,cfg)

cpu


### LRP

In [7]:
import lrp_function as lrp

mask=(hour,2) ## mask=(hour,pollutant) where pollutant is selected from ['NO', 'PM10', 'NO2','O3']
len_static=stat.shape[-1]
lrp_seq2seq= lrp.lrp_decoder_encoder(cfg,len_static, wi_tuples,wh_tuples,bi_tuples,bh_tuples,
                                          wl,bl)

## y: output of forward pass LRP
## Rxf: relevance scores of input sequence forecast
## Rx : relevance scores of input sequence historical
## Ry: relevance scores of intermidiate decoder outputs
## Rstatic: relevance scores of static input (satation number) 
y,Rxf,Rx,Ry,Rstatic=lrp_seq2seq.lrp_seq2seq(inp_p,inp_f,stat,mask,epsilon=0.01,bias_factor=1.0)


sanity decoder
 7.8141885638664732028 in comparison with 7.81418856386647298 
sanityCheckResult True
sanity check encoder
 7.8141885638664732028 in comparison with 7.8141885638664732162 
sanityCheckResult True


#### compare LRP forward pass result  with predictions of the model

In [8]:
model=um.load_model(cfg,device,loading_path)
prediction= model.forward(inp_p,inp_f,stat)

torch.allclose(prediction,y)

cpu


True